# GPX Viewer

This app lets you to display a track from a GPX file recorded with a GPS device.

In [1]:
import sys,os
from ipywidgets import Button, HTML, HBox, FileUpload, Label, Output, Layout, Image

sys.path.append(os.getcwd() + '/src')
import sailing_analytics
import trail_mapping

# session = sailing_analytics.SailingSession("examples/San Mateo Kitefoil.gpx")

# with open("examples/San Mateo Kitefoil.gpx", "r") as f:
#     trail_mapping.plot_gpx(f)

In [2]:
tools = ["voila", "ipyleaflet", "ipywidgets", "bqplot"]
logos = []
for tool in tools:
    with open(f'./img/{tool}.png', 'rb') as f:
        image = f.read()
    img = Image(value=image, format='png',layout=Layout(padding='10px'))
    logos.append(img)
HBox([Label(value='Powered by:')] + logos, layout=Layout(flex_flow='row', align_items='center'))

In [16]:
"""
create the output widget to place the results
"""
out = Output()

In [17]:
def show_uploader():
    uploader = FileUpload(accept='.gpx', multiple=False)

    def handle_upload(change):
        # keep only the last file
        # TODO: check if this should be fixed in FileUpload widget
        # when multiple=False
        *_, (_, f) = change['new'].items()
        gpx_content = f['content'].decode('utf-8')
        out.clear_output()
        with StringIO(gpx_content) as gpx_file:
            with out:
                trail_mapping.plot_gpx(gpx_file)

    uploader.observe(handle_upload, names='value')

    display(uploader)

In [18]:
def show_examples():
    example_folder = "./examples"
    examples = [f for f in os.listdir(example_folder) if f.endswith('.gpx')]
    
    def create_example(name):
        filename = os.path.join(example_folder, name)
        
        @out.capture()
        def on_example_clicked(change):
            out.clear_output()
            with open(filename) as f:
                trail_mapping.plot_gpx(f)
                with out:
                    trail_mapping.plot_gpx(f)
    
        button = Button(description=os.path.splitext(name)[0])
        button.on_click(on_example_clicked)
        return button

    
    buttons = [create_example(example) for example in examples]
    line = HBox(buttons, layout=Layout(flex_flow='row', align_items='center'))
    display(line)

In [19]:
show_uploader()

FileUpload(value={}, accept='.gpx', description='Upload')

If you don't have a GPX file, try with one of the following examples:

In [20]:
show_examples()

In [31]:
def show_map(session):
    def map_button():
        @out.capture()
        def map_button_clicked(change):
            out.clear_output()
            display(out)
            with out:
                print("cats")
    
        button = Button(description="Show Map")
        button.on_click(map_button_clicked)
        return button
    
    display(HBox([map_button()], layout=Layout(flex_flow='row', align_items='center')))
    


In [32]:
# session = sailing_analytics.SailingSession("examples/San Mateo Kitefoil.gpx")
session='sdf'
show_map(session)

In [11]:
# session.map()

In [75]:
# session.windrose()


In [110]:
trail_mapping.plot_gpx(f)

In [30]:
print(help(out))

Help on Output in module ipywidgets.widgets.widget_output object:

class Output(ipywidgets.widgets.domwidget.DOMWidget)
 |  Output(*args: Any, **kwargs: Any) -> Any
 |  
 |  Widget used as a context manager to display output.
 |  
 |  This widget can capture and display stdout, stderr, and rich output.  To use
 |  it, create an instance of it and display it.
 |  
 |  You can then use the widget as a context manager: any output produced while in the
 |  context will be captured and displayed in the widget instead of the standard output
 |  area.
 |  
 |  You can also use the .capture() method to decorate a function or a method. Any output 
 |  produced by the function will then go to the output widget. This is useful for
 |  debugging widget callbacks, for example.
 |  
 |  Example::
 |      import ipywidgets as widgets
 |      from IPython.display import display
 |      out = widgets.Output()
 |      display(out)
 |  
 |      print('prints to output area')
 |  
 |      with out:
 |    